In [ ]:
import sys
print(sys.version_info)

import keras
print(keras.__version__)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

In [ ]:
# global parameters

eq_steps = 256
N = 32
nt = 1000

In [ ]:
# make sure corresponding files are prepared

df_train = pd.read_csv(f'Samples/train_N{N}steps{eq_steps}nt{nt}.csv')
df_test = pd.read_csv(f'Samples/test_N{N}steps{eq_steps}nt{nt}.csv')
df_train.rename(columns={f'{N*N}': 'Phase', f'{N*N+1}': 'InvTemp'}, inplace=True)
df_test.rename(columns={f'{N*N}': 'Phase', f'{N*N+1}': 'InvTemp'}, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.iloc[0:2,0:N*N]

In [ ]:
i = np.random.randint(0, nt//2)
plt.imshow(df_train.iloc[i,0:N*N].values.reshape((N, N)))

In [ ]:
train_lattices = df_train.iloc[:,0:N*N]
train_labels = df_train['Phase']
test_lattices = df_test.iloc[:,0:N*N]
test_labels = df_test['Phase']

In [ ]:
test_lattices.shape

In [ ]:
train_lattices = train_lattices.values.reshape((-1, N, N, 1))
test_lattices = test_lattices.values.reshape((-1, N, N, 1))
input_shape = (N, N, 1)

In [ ]:
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.regularizers import l2

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
network = models.Sequential()
network.add(layers.InputLayer(input_shape))
network.add(layers.Flatten())
network.add(layers.Dense(100, activation='sigmoid', kernel_regularizer=l2(0.01)))
#network.add(layers.Activation('sigmoid'))
network.add(layers.Dense(2, activation='softmax'))
network.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
network.fit(train_lattices, train_labels, epochs=30, batch_size=128)

In [ ]:
test_loss, test_acc = network.evaluate(test_lattices, test_labels)
print('Neural network accuracy on test set:', test_acc)

In [ ]:
#for tSNE
train_lattices_tsne = df_train.iloc[:,0:N*N].values
test_lattices_tsne = df_test.iloc[:,0:N*N].values
train_labels_tsne = df_train['Phase']

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=33)

X_tsne = tsne.fit_transform(train_lattices_tsne)

In [ ]:
# Visualize
plt.figure(figsize=(12,10))

plt.scatter(
    X_tsne[:,0],
    X_tsne[:,1],
    c=train_labels_tsne,
    alpha=0.7,
    s=40,
)

plt.colorbar();  # add colorbar with mapping: color --> class
plt.title('t-SNE for Ising lattices');

In [ ]:
#early workaround
#df_train['Repr'] = df_train['Configuration'].apply(lambda x: np.array(json.loads(x.replace('\n',',').replace('1 ','1, '))))
#df_test['Repr'] = df_test['Configuration'].apply(lambda x: np.array(json.loads(x.replace('\n',',').replace('1 ','1, '))))